In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
import prepare_data
from utils_compare_methods import *
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import vedo
import vedo.dolfin as vdf
from prepare_data import rotate, outside_ball
import pandas as pd

sns.set_theme()
sns.set_context("paper")
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

In [ ]:
nb_data = 1500
save_figs = True

small_data = False  # False=>on utilise toutes les données
level = 2
data = DataLoader(small_data)
agent = Agent(data, small_data)

if not (os.path.exists(f"./images_{nb_data}/")) and save_figs:
    os.makedirs(f"./images_{nb_data}/")

epochs = [100, 200, 500, 750, 1000, 1250, 1500, 2000]
print(len(epochs))
print(epochs)

indices = list(range(0, len(epochs)))
size_per_fig = 4
nb_rows = 2
size_row = int(len(indices) / nb_rows)

In [ ]:
nb_epochs = 2000
agent.model.load_weights(f"./models_{nb_data}/model_{nb_epochs}/model_weights")

In [ ]:
def new_create_FG_numpy(nb_data, nb_vert):
    xy = np.linspace(0.0, 1.0, nb_vert)
    XX, YY = np.meshgrid(xy, xy)
    XX = np.reshape(XX, [-1])
    YY = np.reshape(YY, [-1])
    XXYY = np.stack([XX, YY])

    mu0 = np.random.uniform(0.2, 0.8, size=[nb_data, 1])
    mu1 = np.random.uniform(0.2, 0.8, size=[nb_data, 1])
    sigma = np.random.uniform(0.15, 0.45, size=[nb_data, 1])

    alpha = np.random.uniform(-0.8, 0.8, size=[nb_data, 1])
    beta = np.random.uniform(-0.8, 0.8, size=[nb_data, 1])

    x_0 = np.random.uniform(0.2, 0.8, size=[nb_data, 1])
    y_0 = np.random.uniform(0.2, 0.8, size=[nb_data, 1])
    lx = np.random.uniform(0.2, 0.45, size=[nb_data, 1])
    ly = np.random.uniform(0.2, 0.45, size=[nb_data, 1])
    theta = np.random.uniform(0.0, 0.6, size=[nb_data, 1])
    check_data = 0
    for n in range(nb_data):
        new_generation = 0
        xx_0, yy_0, llx, lly = x_0[n][0], y_0[n][0], lx[n][0], ly[n][0]
        xx0_llxp = rotate([xx_0, yy_0], [xx_0 + llx, yy_0], theta[n])
        xx0_llxm = rotate([xx_0, yy_0], [xx_0 - llx, yy_0], theta[n])
        yy0_llyp = rotate([xx_0, yy_0], [xx_0, yy_0 + lly], theta[n])
        yy0_llym = rotate([xx_0, yy_0], [xx_0, yy_0 - lly], theta[n])
        while (
            (outside_ball(xx0_llxp))
            or (outside_ball(xx0_llxm))
            or (outside_ball(yy0_llyp))
            or (outside_ball(yy0_llym))
        ):
            x_0[n][0] = np.random.uniform(0.2, 0.8, size=[1, 1])[0]
            y_0[n][0] = np.random.uniform(0.2, 0.8, size=[1, 1])[0]
            lx[n][0] = np.random.uniform(0.2, 0.45, size=[1, 1])[0]
            ly[n][0] = np.random.uniform(0.2, 0.45, size=[1, 1])[0]
            xx_0, yy_0, llx, lly = x_0[n][0], y_0[n][0], lx[n][0], ly[n][0]
            xx0_llxp = rotate([xx_0, yy_0], [xx_0 + llx, yy_0], theta[n])
            xx0_llxm = rotate([xx_0, yy_0], [xx_0 - llx, yy_0], theta[n])
            yy0_llyp = rotate([xx_0, yy_0], [xx_0, yy_0 + lly], theta[n])
            yy0_llym = rotate([xx_0, yy_0], [xx_0, yy_0 - lly], theta[n])
            new_generation += 1
        check_data += 1

    for n in range(nb_data):
        new_generation = 0
        xx_0, yy_0, llx, lly, ttheta = (
            x_0[n][0],
            y_0[n][0],
            lx[n][0],
            ly[n][0],
            theta[n][0],
        )
        mmu0, mmu1 = mu0[n][0], mu1[n][0]
        sigma[n][0] = np.random.uniform(min(llx, lly) / 2.0, max(llx, lly))
        while eval_phi(np, mmu0, mmu1, xx_0, yy_0, llx, lly, ttheta) > -0.1:
            mu0[n][0] = np.random.uniform(0.2, 0.8, size=[1, 1])[0]
            mu1[n][0] = np.random.uniform(0.2, 0.8, size=[1, 1])[0]
            mmu0, mmu1 = mu0[n][0], mu1[n][0]

        check_data += 1

    F = call_F(np, XXYY, mu0, mu1, sigma)
    F = np.reshape(F, [nb_data, nb_vert, nb_vert])

    G = call_G(np, XXYY, alpha, beta)
    G = np.reshape(G, [nb_data, nb_vert, nb_vert])

    phi = call_phi(np, XXYY, x_0, y_0, lx, ly, theta)
    phi = np.reshape(phi, [nb_data, nb_vert, nb_vert])

    params = np.concatenate(
        [mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta], axis=1
    )
    return F, phi, G, params

In [ ]:
def compare_std_phi_fem_and_fno(param, Plot=False, epoch=2000):
    u_ex, V_ex, dx_ex = compute_standard_fem(64, param, True)

    solver = PhiFemSolver_error(nb_cell=64 - 1, params=param)
    (
        u_phi_fem,
        V_phi_fem,
        cell_selection_phi,
        submesh_construction_phi,
        ghost_cell_selection_phi,
        resolution_time_phi,
    ) = solver.solve_one(0)
    u_phi_fem_proj = df.project(
        u_phi_fem, V_ex, solver_type="gmres", preconditioner_type="hypre_amg"
    )
    l2_error_phi_fem = (
        df.assemble((((u_ex - u_phi_fem_proj)) ** 2) * dx_ex) ** (0.5)
    ) / (df.assemble((((u_ex)) ** 2) * dx_ex) ** (0.5))

    (
        u_std,
        construction_time_standard,
        resolution_time_standard,
    ) = compute_standard_fem(64, param)
    u_std_fem_proj = df.project(
        u_std, V_ex, solver_type="gmres", preconditioner_type="hypre_amg"
    )
    l2_error_std_fem = (
        df.assemble((((u_ex - u_std_fem_proj)) ** 2) * dx_ex) ** (0.5)
    ) / (df.assemble((((u_ex)) ** 2) * dx_ex) ** (0.5))

    mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta = param
    agent.model.load_weights(f"./models_{nb_data}/model_{epoch}/model_weights")

    phi = generate_phi_numpy(x_0, y_0, lx, ly, theta, 64)
    F = generate_F_numpy(mu0, mu1, sigma, 64) / data.max_norm_F
    G = generate_G_numpy(alpha, beta, 64)
    X = generate_manual_new_data_numpy(phi, F, G)
    start_call = time.time()
    Y = agent.model.call(X)
    end_call = time.time()
    temps_fno = end_call - start_call
    solution_predite = X[:, :, :, 1] * Y[:, :, :, 0] + X[:, :, :, 2]
    solution_predite = np.reshape(solution_predite, (64, 64))
    sol_predite_fenics = convert_numpy_matrix_to_fenics(
        solution_predite, 64, 1
    )
    sol_predite_fenics_proj_V_ex = df.project(
        sol_predite_fenics,
        V_ex,
        solver_type="gmres",
        preconditioner_type="hypre_amg",
    )

    l2_error_fno = (
        df.assemble((((u_ex - sol_predite_fenics_proj_V_ex)) ** 2) * dx_ex)
        ** (0.5)
    ) / (df.assemble((((u_ex)) ** 2) * dx_ex) ** (0.5))

    if Plot:
        plt.figure(figsize=(16, 4))

        ax1 = plt.subplot(141)
        img = df.plot(u_ex, mode="color", cmap=my_cmap)
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes("bottom", size="5%", pad=0.3)
        ax1.grid(False)
        ax1.set_title("Exact solution", fontsize=15)
        plt.colorbar(img, cax=cax, orientation="horizontal")

        ax2 = plt.subplot(142)
        img = df.plot(u_phi_fem_proj, mode="color", cmap=my_cmap)
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes("bottom", size="5%", pad=0.3)
        ax2.grid(False)
        ax2.set_title(
            "$\phi$-FEM solution \n$L^2$ relative error : "
            + f"{l2_error_phi_fem:.5f}",
            fontsize=15,
        )
        plt.colorbar(img, cax=cax, orientation="horizontal")

        ax3 = plt.subplot(143)
        img = df.plot(u_std_fem_proj, mode="color", cmap=my_cmap)
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes("bottom", size="5%", pad=0.3)
        ax3.grid(False)
        ax3.set_title(
            "Standard FEM solution \n$L^2$ relative error : "
            + f"{l2_error_std_fem:.5f}",
            fontsize=15,
        )
        plt.colorbar(img, cax=cax, orientation="horizontal")

        ax4 = plt.subplot(144)
        img = df.plot(sol_predite_fenics_proj_V_ex, mode="color", cmap=my_cmap)
        divider = make_axes_locatable(ax4)
        cax = divider.append_axes("bottom", size="5%", pad=0.3)
        ax4.grid(False)
        ax4.set_title(
            "Predicted solution \n$L^2$ relative error : "
            + f"{l2_error_fno:.5f}",
            fontsize=15,
        )
        plt.colorbar(img, cax=cax, orientation="horizontal")

        plt.tight_layout(pad=1.3)
        if Plot and save_figs:
            plt.savefig(f"./images_{nb_data}/example_output_FEMs_FNO.png")
        plt.show()

    return (
        l2_error_phi_fem,
        l2_error_std_fem,
        l2_error_fno,
        cell_selection_phi,
        submesh_construction_phi,
        ghost_cell_selection_phi,
        resolution_time_phi,
        construction_time_standard,
        resolution_time_standard,
        temps_fno,
    )

In [ ]:
# (mu0, mu1, sigma, x_0, y_0, lx, ly, theta) = [ 0.58894387,  0.45582733 , 0.24063726 , 0.60009858  ,0.4920412,   0.28568717 , 0.24311015 , 0.0917393,  -0.42856707 ,-0.34318572]
F, phi, G, params = new_create_FG_numpy(10, 64)
params = [
    [
        0.58894387,
        0.45582733,
        0.24063726,
        0.60009858,
        0.4920412,
        0.28568717,
        0.24311015,
        0.0917393,
        -0.42856707,
        -0.34318572,
    ]
]
i = 0
for param in params:
    i += 1
    for epoch in [2000]:
        print(f"######  {epoch=}  Param : {i}/{np.shape(params)[0]} ######")
        (
            l2_error_phi_fem,
            l2_error_std_fem,
            l2_error_fno,
            cell_selection_phi,
            submesh_construction_phi,
            ghost_cell_selection_phi,
            resolution_time_phi,
            construction_time_standard,
            resolution_time_standard,
            temps_fno,
        ) = compare_std_phi_fem_and_fno(param, True, epoch=epoch)
        print(f"{l2_error_fno=}")

In [ ]:
def compute_errors_multiple_epochs(epochs, params):
    u_exs, V_exs, dx_exs = [], [], []
    L2_error_phi_fem, L2_error_std_fem = [], []
    Temps_phi, Temps_std = [], []
    index = 1
    for param in params:
        print(f"Call FEMS param : {index} / {len(params)}")
        u_ex, V_ex, dx_ex = compute_standard_fem(64, param, True)
        u_exs.append(u_ex)
        V_exs.append(V_ex)
        dx_exs.append(dx_ex)
        solver = PhiFemSolver_error(nb_cell=64 - 1, params=param)
        (
            u_phi_fem,
            V_phi_fem,
            cell_selection_phi,
            submesh_construction_phi,
            ghost_cell_selection_phi,
            resolution_time_phi,
        ) = solver.solve_one(0)
        temps_phi = [
            cell_selection_phi,
            submesh_construction_phi,
            ghost_cell_selection_phi,
            resolution_time_phi,
        ]
        u_phi_fem_proj = df.project(
            u_phi_fem,
            V_ex,
            solver_type="gmres",
            preconditioner_type="hypre_amg",
        )
        l2_error_phi_fem = (
            df.assemble((((u_ex - u_phi_fem_proj)) ** 2) * dx_ex) ** (0.5)
        ) / (df.assemble((((u_ex)) ** 2) * dx_ex) ** (0.5))

        (
            u_std,
            construction_time_standard,
            resolution_time_standard,
        ) = compute_standard_fem(64, param)
        u_std_fem_proj = df.project(
            u_std, V_ex, solver_type="gmres", preconditioner_type="hypre_amg"
        )
        l2_error_std_fem = (
            df.assemble((((u_ex - u_std_fem_proj)) ** 2) * dx_ex) ** (0.5)
        ) / (df.assemble((((u_ex)) ** 2) * dx_ex) ** (0.5))

        temps_std = [construction_time_standard, resolution_time_standard]
        L2_error_phi_fem.append(l2_error_phi_fem)
        L2_error_std_fem.append(l2_error_std_fem)
        Temps_phi.append(temps_phi)
        Temps_std.append(temps_std)
        index += 1

    errors_fno, times_fno = [], []

    for j in epochs:
        print(f"Epoch : {j}")
        agent.model.load_weights(f"./models_{nb_data}/model_{j}/model_weights")
        L2_error_fno = []
        Temps_fno = []
        for i in range(len(params)):
            mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta = params[i]
            phi = generate_phi_numpy(x_0, y_0, lx, ly, theta, 64)
            F = generate_F_numpy(mu0, mu1, sigma, 64) / data.max_norm_F
            G = generate_G_numpy(alpha, beta, 64)
            X = generate_manual_new_data_numpy(phi, F, G)
            start_call = time.time()
            Y = agent.model.call(X)
            end_call = time.time()
            temps_fno = end_call - start_call
            solution_predite = (
                X[:, :, :, 1] * Y[:, :, :, 0] + X[:, :, :, 2]
            )  # * X[:, :, :, -1]
            solution_predite = np.reshape(solution_predite, (64, 64))
            sol_predite_fenics = convert_numpy_matrix_to_fenics(
                solution_predite, 64, 1
            )
            sol_predite_fenics_proj_V_ex = df.project(
                sol_predite_fenics,
                V_exs[i],
                solver_type="gmres",
                preconditioner_type="hypre_amg",
            )

            l2_error_fno = (
                df.assemble(
                    (((u_exs[i] - sol_predite_fenics_proj_V_ex)) ** 2)
                    * dx_exs[i]
                )
                ** (0.5)
            ) / (df.assemble((((u_exs[i])) ** 2) * dx_exs[i]) ** (0.5))
            L2_error_fno.append(l2_error_fno)
            Temps_fno.append(temps_fno)
        errors_fno.append(L2_error_fno)
        times_fno.append(Temps_fno)

    return (
        L2_error_phi_fem,
        L2_error_std_fem,
        errors_fno,
        Temps_phi,
        Temps_std,
        times_fno,
    )

In [ ]:
F, phi, G, params = new_create_FG_numpy(300, 64)

In [ ]:
(
    L2_error_phi_fem,
    L2_error_std_fem,
    errors_fno,
    Temps_phi,
    Temps_std,
    times_fno,
) = compute_errors_multiple_epochs(epochs, params)

In [ ]:
colors = sns.color_palette("mako").as_hex()
j, k = 0, 0
fig, axes = plt.subplots(
    nb_rows,
    size_row,
    figsize=(size_per_fig * size_row, size_per_fig * nb_rows),
)

for i in range(len(indices)):
    if k == size_row:
        k = 0
        j += 1
    sns.histplot(
        data=errors_fno[indices[i]],
        kde=True,
        bins=20,
        color=colors[2],
        edgecolor="k",
        log_scale=True,
        label="FNO " + str(epochs[indices[i]]) + " epochs",
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    sns.histplot(
        data=L2_error_phi_fem,
        kde=True,
        bins=10,
        color=colors[3],
        edgecolor="k",
        log_scale=True,
        label="$\phi$-FEM",
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    sns.histplot(
        data=L2_error_std_fem,
        kde=True,
        bins=10,
        color=colors[1],
        edgecolor="k",
        log_scale=True,
        label="Standard FEM",
        stat="proportion",
        legend=True,
        ax=axes[j, k],
    )
    axes[j, k].set_xlim(1e-4, 1e0)
    axes[j, k].set_ylim(0.0, 0.22)
    axes[j, k].set_xlabel("$L^2$ relative error")
    axes[j, k].legend()
    k += 1

plt.tight_layout()
if save_figs:
    plt.savefig(
        f"./images_{nb_data}/histograms_new_data_compare_methods_L2.png"
    )
plt.show()

In [ ]:
error_tab = []
error_tab.append(L2_error_phi_fem)
error_tab.append(L2_error_std_fem)

plot_errors_fno = [7]
new_fno_errors = [errors_fno[i] for i in plot_errors_fno]
plot_epochs = [epochs[i] for i in plot_errors_fno]
error_tab += new_fno_errors

abs_str = ["$\phi$-FEM", "Standard FEM"]
abs_str += ["FNO " + str(epochs[i]) + " epochs" for i in plot_errors_fno]
errors = np.array(error_tab[:])
print(np.shape(errors))
dataframe = pd.DataFrame(errors.transpose(), columns=abs_str)

plt.figure(figsize=(10, 5))

sns.boxplot(data=dataframe, palette="ch:s=.25,rot=-.25")
plt.yscale("log")
plt.xlabel("Method", fontsize=15)
plt.ylabel("$L^2$ Relative error", fontsize=15)
if save_figs:
    plt.savefig(f"./images_{nb_data}/boxplots_new_data_compare_method.png")
plt.show()

In [ ]:
epochs_mean = list(i * 50 for i in range(1, 41))

(
    L2_error_phi_fem,
    L2_error_std_fem,
    errors_fno,
    Temps_phi,
    Temps_std,
    times_fno,
) = compute_errors_multiple_epochs(epochs_mean, params)

In [ ]:
means = np.mean(errors_fno, axis=1)
standard_deviation = np.std(errors_fno, axis=1)
maxs = np.max(errors_fno, axis=1)
mins = np.min(errors_fno, axis=1)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.semilogy(epochs_mean, means, "-+", label="Mean")
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.semilogy(epochs_mean, standard_deviation, "-+", label="Standard deviation")
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.grid(True, "both", "both")
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(epochs_mean, maxs, "-+", label="Maximum")
plt.semilogy(epochs_mean, mins, "-+", label="Minimum")
plt.grid(True, "both", "both")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("$L^2$ relative error")
plt.tight_layout()

if save_figs:
    plt.savefig(
        f"./images_{nb_data}/min_mean_max_error_epochs_new_data_compare_methods.png"
    )
plt.show()